# Conversion of TXT to MAT for gm/ID result files for SG13G2

**(c) 2024 Boris Murmann and Harald Pretl**

Licensed under the Apache License, Version 2.0 (the “License”); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

In [1]:
import pandas as pd
import numpy as np
from numpy.core.records import fromarrays
from scipy.io import savemat

In [2]:
choice = 0  #start from 0
devices = ['sg13_lv_nmos', 'sg13_lv_pmos']

# widths used for characterization and fringe cap parameters (fringe caps are not included in ngspice output)
w = np.array([5, 5])
nfing = np.array([1, 1])

In [3]:
df_raw = pd.read_csv('techsweep_'+devices[choice]+'.txt', sep=r'\s+')
par_names = df_raw.columns.to_list()
par_prefix = par_names[1].split('[')[0]

In [4]:
# remove extra headers in file body and unwanted columns
df = df_raw.drop(['frequency', 'frequency.1'], axis=1)
df = df.apply(pd.to_numeric)

# rename columns for readability
df.columns = df.columns.str.removeprefix(par_prefix+'[')
df.columns = df.columns.str.replace(par_prefix[1:], '')
df.columns = df.columns.str.removesuffix(']')

# round sweep vectors to easily addressable values
df['l'] = df['l'].apply(lambda x: round(x*1e6, 3))
df['vgs'] = df['vgs'].apply(lambda x: round(x, 3))
df['vds'] = df['vds'].apply(lambda x: round(x, 3))
df['vsb'] = df['vsb'].apply(lambda x: round(x, 3))
df

119072 rows × 24 columns

In [5]:
# sweep variable vectors
l = np.unique(abs(df['l']))
vgs = np.unique(abs(df['vgs']))
vds = np.unique(abs(df['vds']))
vsb = np.unique(abs(df['vsb']))

In [6]:
# data
# ngspice sweep order is l, vgs, vds, vsb
dims = [len(l), len(vgs), len(vds), len(vsb)]
id = np.reshape(df['ids'].values, dims, order='C')
vt = np.reshape(df['vth'].values, dims, order='C')
gm = np.reshape(df['gm'].values, dims, order='C')
gmb = np.reshape(df['gmb'].values, dims, order='C')
gds = np.reshape(df['gds'].values, dims, order='C')
cgsol = np.reshape(df['cgsol'].values, dims, order='C')
cgg = np.reshape(df['cgg'].values, dims, order='C') \
      + np.reshape(df['cgdol'].values, dims, order='C') \
      + np.reshape(df['cgsol'].values, dims, order='C')
cgb = -np.reshape(df['cgb'].values, dims, order='C')
cgd = -np.reshape(df['cgd'].values, dims, order='C') \
      + np.reshape(df['cgdol'].values, dims, order='C')
cgs = -np.reshape(df['cgs'].values, dims, order='C') \
      + np.reshape(df['cgsol'].values, dims, order='C')
cdd = np.reshape(df['cdd'].values, dims, order='C') \
      + np.reshape(df['cjd'].values, dims, order='C') \
      + np.reshape(df['cgdol'].values, dims, order='C')
css = np.reshape(df['css'].values, dims, order='C') \
      + np.reshape(df['cjs'].values, dims, order='C') \
      + np.reshape(df['cgsol'].values, dims, order='C')
sth = np.reshape(df['sid'].values, dims, order='C')
sfl = np.reshape(df['sfl'].values, dims, order='C')

In [7]:
dic = {
  "INFO": "IHP SG13G2, 130nm CMOS, PSP",
  "CORNER": "NOM",
  "TEMP": 300.0,
  "VGS": vgs,
  "VDS": vds,
  "VSB": vsb,
  "L": l,
  "W": w[choice],
  "NFING": nfing[choice],
  "ID": id,
  "VT": vt,
  "GM": gm,
  "GMB": gmb,
  "GDS": gds,
  "CGG": cgg,
  "CGB": cgb,
  "CGD": cgd,
  "CGS": cgs,
  "CDD": cdd,
  "CSS": css,
  "STH": sth,
  "SFL": sfl
}

savemat(devices[choice]+'.mat', {devices[choice]: dic})